In [2]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

def get_url(position,location):
    """Generate a url from position and location"""
    template='https://www.indeed.com/jobs?q={}&l={}'
    url=template.format(position, location)
    return url

def job_summary(card):
    job_summary=""
    try:
        for i in card.find_all('li'):
            job_summary+=i.text
    except AttributeError:
        job_summary=x.find_all('li').text
    return job_summary

def job_rating(card):
    try:
        job_rating=card.find('span','ratingsDisplay').a.text
    except AttributeError:
        job_rating=''
    return job_rating

def job_salary(card):
    try:
        job_salary=card.find('div','metadata salary-snippet-container').text
    except AttributeError:
        job_salary=''
    return job_salary

def job_type(card):
    try:
        job_type=card.find('div','metadata').find('div','attribute_snippet').text
    except AttributeError:
        job_type=''
    return job_type

def post_days(card):
    try:
        post_days=card.find("span","date").contents[1].text
    except IndexError:
        post_days=''
    return post_days

def get_record(card):
    title_name=card.h2.text
    company_name=card.find("div",'heading6').contents[0].text
    location=card.find('div','companyLocation').text
    summary=job_summary(card)
    post_day=post_days(card)
    rating=job_rating(card)  
    salary=job_salary(card)
    jobs_type=job_type(card)
    record=(title_name,company_name,location,summary,post_day,
           rating,salary,jobs_type)
    return record

def main(position,location):
    records=[]
    url=get_url(position,location)
    while True:
        response=requests.get(url)
        soup=BeautifulSoup(response.text,'html.parser')
        cards=soup.find_all('div','job_seen_beacon')
        try:
            url='https://www.indeed.com'+soup.find('a',{'aria-label':'Next'}).get('href')
        except AttributeError:
            break

        response=requests.get(url)
        soup=BeautifulSoup(response.text,'html.parser')
        cards=soup.find_all('div','job_seen_beacon')
        
        for card in cards:
            record=get_record(card)
            records.append(record)

    with open('results.csv','w',newline='',encoding='utf-8') as f:
        writer=csv.writer(f)
        writer.writerow(['title_name','company_name','location','summary','post_day',
               'rating','salary','jobs_type'])
        writer.writerows(records)

In [6]:
main('data science','Houston')